# Tabular data preprocessing

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.tabular import * 
from fastai.docs import *

## Overview

This package contains the basic class to define a transformation for preprocessing dataframes of tabular data, as well as basic [`TabularTransform`](/tabular.transform.html#TabularTransform). Preprocessing includes things like
- replacing non-numerical variables by categories, then their ids,
- filling missing values,
- normalizing continuous variables.

In all those steps we have to be careful to use the correspondance we decide on our training set (which id we give to each category, what is the value we put for missing data, or how the mean/std we use to normalize) on our validation or test set. To deal with this, we use a speciall class called [`TabularTransform`](/tabular.transform.html#TabularTransform).

The data used in this document page is a subset of the [adult dataset](https://archive.ics.uci.edu/ml/datasets/adult). It gives a certain amount of data on individuals to train a model to predict wether their salary is greater than \$50k or not.

In [ ]:
df = get_adult()
train_df, valid_df = df[:800].copy(),df[800:].copy()
train_df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,>=50k
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,1
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,1
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,0
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,1
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,0


We see it contains numerical variables (like `age` or `education-num`) as well as categorical ones (like `workclass` or `relationship`). The original dataset is clean, but we removed a few values to give examples of dealing with missing variables.

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
cont_names = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

## Transforms for tabular data

In [ ]:
show_doc(TabularTransform, doc_string=False)

## <a id=TabularTransform></a>`class` `TabularTransform`
> `TabularTransform`(`cat_names`:`StrList`, `cont_names`:`StrList`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L7">[source]</a>

Base class for creating transforms for dataframes with categorical variables `cat_names` and continuous variables `cont_names`. Note that any column not in one of those lists won't be touched.

In [ ]:
show_doc(TabularTransform.__call__)

#### <a id=__call__></a>`__call__`
> `__call__`(`df`:`DataFrame`, `test`:`bool`=`False`)


Apply the correct function to `df` depending on `test`. <a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L12">[source]</a>

This simply calls `apply_test` if `test` or `apply_train` otherwise. Those functions apply the changes in place.

In [ ]:
show_doc(Categorify.apply_train, doc_string=False)

#### <a id=apply_train></a>`apply_train`
> `apply_train`(`df`:`DataFrame`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L27">[source]</a>

Must be implemented by an inherited class with the desired transformation logic.

In [ ]:
show_doc(Categorify.apply_test, doc_string=False)

#### <a id=apply_test></a>`apply_test`
> `apply_test`(`df`:`DataFrame`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L33">[source]</a>

If not implemented by an inherited class, defaults to calling `apply_train`.

The following [`TabularTransform`](/tabular.transform.html#TabularTransform) are implemented in the fastai library. Note that the replacement from categories to codes as well as the normalization of continuous variables are automatically done in a [`TabularDataset`](/tabular.data.html#TabularDataset).

In [ ]:
show_doc(Categorify, doc_string=False)

## <a id=Categorify></a>`class` `Categorify`
> `Categorify`(`cat_names`:`StrList`, `cont_names`:`StrList`) :: [`TabularTransform`](/tabular.transform.html#TabularTransform)
<a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L24">[source]</a>

Changes the categorical variables in `cat_names` in categories. Variables in `cont_names` aren't affected.

In [ ]:
show_doc(Categorify.apply_train, doc_string=False)

#### <a id=apply_train></a>`apply_train`
> `apply_train`(`df`:`DataFrame`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L27">[source]</a>

Transforms the variable in the `cat_names` columns in categories. The category codes are the unique values in these columns.

In [ ]:
show_doc(Categorify.apply_test, doc_string=False)

#### <a id=apply_test></a>`apply_test`
> `apply_test`(`df`:`DataFrame`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L33">[source]</a>

Transforms the variable in the `cat_names` columns in categories. The category codes are the ones used for the training set, new categories are replaced by NaN. 

In [ ]:
tfm = Categorify(cat_names, cont_names)
tfm(train_df)
tfm(valid_df, test=True)

Since we haven't changed the categories by their codes, nothing visible has changed in the dataframe yet, but we can check that the variables are now categorical and view their corresponding codes.

In [ ]:
train_df['workclass'].cat.categories

Index([' ?', ' Federal-gov', ' Local-gov', ' Private', ' Self-emp-inc',
       ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
      dtype='object')

The test set will be given the same category codes as the training set.

In [ ]:
valid_df['workclass'].cat.categories

Index([' ?', ' Federal-gov', ' Local-gov', ' Private', ' Self-emp-inc',
       ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
      dtype='object')

In [ ]:
show_doc(FillMissing, doc_string=False)

## <a id=FillMissing></a>`class` `FillMissing`
> `FillMissing`(`cat_names`:`StrList`, `cont_names`:`StrList`, `fill_strategy`:[`FillStrategy`](/tabular.transform.html#FillStrategy)=`<FillStrategy.MEDIAN: 1>`, `add_col`:`bool`=`True`, `fill_val`:`float`=`0.0`) :: [`TabularTransform`](/tabular.transform.html#TabularTransform)
<a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L40">[source]</a>

Transform that fills the missing values in `cont_names`. `cat_names` variables are left untouched (their missing value will be raplced by code 0 in the [`TabularDataset`](/tabular.data.html#TabularDataset)). `fill_strategy` is adopted to replace those nans and if `add_col` is True, whenever a column `c` has missing values, a column named `c_nan` is added and flags the line where the value was missing. The `fill_strategy` can be:
- `FillStrategy.MEDIAN`: nans are replaced by the median value of the column,
- `FillStrategy.COMMON`: nans are replaced by the most common value of the column,
- `FillStrategy.CONSTANT`: nans are replaced by `fill_val`.

In [ ]:
show_doc(FillMissing.apply_train, doc_string=False)

#### <a id=apply_train></a>`apply_train`
> `apply_train`(`df`:`DataFrame`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L46">[source]</a>

Fills the missing values in the `cont_names` columns.

In [ ]:
show_doc(FillMissing.apply_test, doc_string=False)

#### <a id=apply_test></a>`apply_test`
> `apply_test`(`df`:`DataFrame`)
<a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L59">[source]</a>

Fills the missing values in the `cont_names` columns with the ones picked during train.

In [ ]:
train_df[cont_names].head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,49,101320,12.0,0,1902,40
1,44,236746,14.0,10520,0,45
2,38,96185,NaN,0,0,32
3,38,112847,15.0,0,0,40
4,42,82297,NaN,0,0,50


In [ ]:
tfm = FillMissing(cat_names, cont_names)
tfm(train_df)
tfm(valid_df, test=True)
train_df[cont_names].head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
0,49,101320,12.0,0,1902,40
1,44,236746,14.0,10520,0,45
2,38,96185,10.0,0,0,32
3,38,112847,15.0,0,0,40
4,42,82297,10.0,0,0,50


Values issing in the `education-num` column are replaced by 10, which is the median of the column in `train_df`. Categorical variables are not changed, since `nan` is simply used as another category.

In [ ]:
valid_df[cont_names].head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
800,45,96975,10.0,0,0,40
801,46,192779,10.0,15024,0,60
802,36,376455,10.0,0,0,38
803,25,50053,10.0,0,0,45
804,37,164526,10.0,0,0,40


## Undocumented Methods - Methods moved below this line will intentionally be hidden

## New Methods - Please document or move to the undocumented section

In [ ]:
show_doc(TabularTransform.apply_train)

#### <a id=apply_train></a>`apply_train`
> `apply_train`(`df`:`DataFrame`)


Function applied to `df` if it's the train set. <a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L17">[source]</a>

In [ ]:
show_doc(TabularTransform.apply_test)

#### <a id=apply_test></a>`apply_test`
> `apply_test`(`df`:`DataFrame`)


Function applied to `df` if it's the test set. <a href="https://github.com/fastai/fastai/blob/master/fastai/tabular/transform.py#L20">[source]</a>